In [ ]:
# Standard library imports
from itertools import product

# Related third party imports
import holoviews as hv
import kwant
from kwant.continuum.discretizer import discretize
import numpy as np
import pandas as pd

# Local imports
import funcs
from funcs import constants, discretized_hamiltonian, lat_from_temp, get_cuts, add_vlead, hopping_between_cuts

%matplotlib inline
hv.notebook_extension()

def plot_CPR(syst, hopping, params, tol=0.01, max_frequencies=1000):
    phases = np.linspace(-np.pi, np.pi, 51)
    H_0_cache = []
    I = [funcs.current_at_phase(syst, hopping, params, H_0_cache, phase, tol, max_frequencies)
                  for phase in phases]

    return hv.Curve((phases, I), kdims=['phase'], vdims=['$I$'], label='Nummerical CPR')

# 1D

In [ ]:
syst, hopping = funcs.make_1d_wire(a=10, L=100, L_sc=100)

params = dict(T=0.06, Delta=0.250, mu=15, k=constants.k,
              hbar=constants.hbar, m_eff=constants.m_eff, current_unit=constants.current_unit, c=constants.c, 
              B_x=0, B_y=0, B_z=0, g=0, mu_B=0, alpha=0, V=lambda x:0)

plot_CPR(syst, hopping, params)

# Simpler way of calculating the super current (but slightly slower)

In [ ]:
from funcs import matsubara_frequency

def null_G(syst, params, n):
    en = matsubara_frequency(n, params)
    gf = kwant.greens_function(syst, en, out_leads=[0], in_leads=[0],
                               check_hermiticity=False, params=params)
    return gf.data[::2, ::2]

def current_from_G_0(G_0s, H12, phase, params):
    t = H12 * np.exp(1j * phase)
    dim = t.shape[0]
    I = 0
    for G_0 in G_0s:
        V = np.zeros_like(G_0, dtype=complex)
        v = t - H12
        V[:dim, dim:] = v.T.conj()
        V[dim:, :dim] = v
        gf = np.linalg.solve(np.identity(2*dim) - G_0 @ V, G_0)
        H12G21 = t.T.conj() @ gf[dim:, :dim]
        H21G12 = t @ gf[:dim, dim:]
        I += -4 * params['T'] * params['current_unit'] * (np.trace(H21G12) - np.trace(H12G21)).imag
    return I

matsfreqs = 500
G_0s = [null_G(syst, params, n) for n in range(matsfreqs)]
H12 = hopping(syst, params=params)
phases = np.linspace(-np.pi, np.pi, 51)
I = [current_from_G_0(G_0s, H12, phase, params) for phase in phases]
hv.Curve((phases, I), kdims=['phase'], vdims=['$I$'])

# Test 2D system and compare with exact equation

In [ ]:
Y = 2
syst, hopping = funcs.make_2d_test_system(Y=Y)
kwant.plot(syst);

params = dict(T=0.0001, hbar=constants.hbar, m=constants.m_eff, 
              Delta=0.001, mu=constants.t*4, k=constants.k, current_unit=constants.current_unit, c=constants.c)
kwant.plotter.bands(syst.leads[1], params=params);

# Analytical comparison
phases = np.linspace(-np.pi, np.pi, 51)
N_modes = Y
tanh_part = np.tanh(params['Delta'] / (2 * params['T'] * params['k']) * np.sqrt(1 - np.sin(0.5 * phases)**2))
sum_part = np.sin(phases) / np.sqrt(1 - np.sin(0.5 * phases)**2)
prefactor = constants.eV * (params['Delta'] * constants.meV) / (2 * constants.hbar) # Delta in J
I_analytical = prefactor * N_modes * sum_part * tanh_part * 1e9 # Eq. (35) from arXiv:cond-mat/0406127v2

(plot_CPR(syst, hopping, params, tol=0.001) *
 hv.Curve((phases, I_analytical), kdims=['phase'], vdims=['$I$'], label='Analytical result'))


# 3D test system

In [ ]:
sx_ = np.array([[0., 1.], [1., 0.]])
sz_ = np.array([[1., 0.], [0., -1.]])

onsite = lambda site, mu: -mu * sz_
onsite_lead = lambda site, mu, delta: -mu * sz_ + delta * sx_
hops = lambda site1, site2, t: -t * sz_

def make_test_system(X, Y, Z):
    lat = kwant.lattice.general(np.eye(3), norbs=2)
    syst = kwant.Builder()
    syst[(lat(x, y, z) for x in range(X) for y in range(Y) for z in range(Z))] = onsite
    syst[lat.neighbors()] = hops

    cuts = get_cuts(syst, lat)
    syst = add_vlead(syst, lat, *cuts)

    lead = kwant.Builder(kwant.TranslationalSymmetry((-1, 0, 0)))
    lead[(lat(0, y, z) for y in range(Y) for z in range(Z))] = onsite_lead
    lead[lat.neighbors()] = hops
    syst.attach_lead(lead)
    syst.attach_lead(lead.reversed())
    syst = syst.finalized()
    
    hopping = hopping_between_cuts(syst, *cuts)
    
    return syst, hopping

delta = 0.01
params = dict(mu=0., t=1., delta=delta, k=1, current_unit=1, T=delta/100)
syst, hopping = make_test_system(X=3, Y=3, Z=3)
kwant.plot(syst);
plot_CPR(syst, hopping, params)

# 3D system with Discretizer Hamiltonian, squared system

In [ ]:
# Test Hamiltonian
syst, hopping = funcs.make_3d_test_system(X=5, Y=3, Z=3, a=1, test_hamiltonian=True)
params = dict(T=0.01, Delta=1, mu=2, t=1, k=1, current_unit=1, c=1)
kwant.plotter.bands(syst.leads[2], params=params)
plot_CPR(syst, hopping, params)

In [ ]:
# Full Hamiltonian
syst, hopping = funcs.make_3d_test_system(X=20, Y=30, Z=30, a=10, test_hamiltonian=False)
params = dict(T=1, Delta=0.250, mu=30, k=constants.k,
              hbar=constants.hbar, m_eff=constants.m_eff, current_unit=constants.current_unit, c=constants.c, 
              B_x=0, B_y=0, B_z=0, g=0, mu_B=0, alpha=0, V=lambda x:0)
kwant.plot(syst);
kwant.plotter.bands(syst.leads[2], params=params);
plot_CPR(syst, hopping, params)

# 3D full wire

In [ ]:
syst_pars = dict(a=8, angle=0, site_disorder=True, L=80, L_sc=8,
                 phi=135, r1=50, r2=70, shape='circle', with_leads=True,
                 with_shell=True, with_vlead=False, holes=True)
syst, hopping = funcs.make_3d_wire(**syst_pars)

kwant.plot(syst)

# 1D Supplementary figure
This is calculated using a cluster because of the many parameters.

In [ ]:
from hpc05 import Client
client = Client(profile='pbs', timeout=10)

In [ ]:
dview = client[:]
dview.use_dill()
lview = client.load_balanced_view()
len(dview)

In [ ]:
%px import sys, os; sys.path.append(os.path.expanduser('~/Work/nanowire_current/'))
%px import funcs, kwant

In [ ]:
syst_pars = dict(a=10, L=500, L_sc=100)
params = dict(B_y=0, B_z=0, Delta=0.25, g=50, mu_B=constants.mu_B, t=constants.t, V=lambda x: 0, T=1, 
              k=constants.k, current_unit=constants.current_unit, c=constants.c, hbar=constants.hbar,
              m_eff=constants.m_eff)
alphas = np.linspace(0, 100, 20)
B_xs = np.linspace(0, 1.5, 20)
mus = [0.1, 0.3 ,1, 3, 10, 30]
vals = list(product(alphas, B_xs, mus))

def fun(x, syst_pars=syst_pars, params=params):
    import kwant, funcs
    syst, hopping = funcs.make_1d_wire(**syst_pars)
    params = dict(**params, alpha=x[0], B_x=x[1], mu=x[2])
    return funcs.I_c(syst, hopping, params)

current_phase = lview.map_async(fun, vals)

current_phase.wait_interactive()

current_phase = current_phase.result()

df = pd.concat((pd.DataFrame(vals, columns=['alpha', 'B_x', 'mu']), pd.DataFrame(current_phase)), axis=1)
df = df.assign(**syst_pars, **params).assign(**constants.__dict__)
df = df.assign(git_hash=funcs.get_git_revision_hash())
df.to_hdf('1d_alpha_vs_B_x.hdf', 'all_data', mode='w')

In [ ]:
def Image(group, x, y, z):
    bounds = (df[x].min(), df[y].min(), df[x].max(), df[y].max())
    table = group.pivot_table(index=[x], columns=[y], values=[z])
    return hv.Image(np.rot90(table), kdims=[x, y], vdims=[z], bounds=bounds)

In [ ]:
%%opts Image (cmap='viridis') [aspect='square'] Layout {+axiswise}
df = pd.read_hdf('data/1d_alpha_vs_B_x.hdf')
ims = [(mu, Image(group, 'B_x', 'alpha', 'current_c')) for mu, group in df.groupby('mu')]
hm = hv.HoloMap(ims, kdims=['mu'])
hm.layout('mu').cols(3)